<a href="https://colab.research.google.com/github/ieg-dhr/DigiKAR/blob/main/JupyterNotebooks_DigiKAR/Factoids_Step2c_VerticalConsolidation_Staatskalender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a script for consolidating factoid lists in AP3.

The package mainly uses the Pandas package in Python to read and manipulate EXCEL data as DataFrames. DataFrames are 2-dimensional data representations in rows and columns. They can be written to different file formats such as CSV, EXCEL, JSON or RDF.

First of all, we need to connect this Colab notebook with your Google Drive and define the directory for input and output data.


In [1]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

Mounted at /content/drive


In the second step, we have to install additional Packages needed for working with CSV, EXCEL and DataFrames.

In [2]:
## install packages that are not part of Python's standard distribution

!pip install xlsxwriter
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 4.7 MB/s eta 0:00:00


In **step 1**, we can import the packages to the script and load our data. Before merging the input files, names will be normalised as some have access spaces, capitalised surnames, or inverted first and last names.

The combined data will be written to a new dataframe and displayed.

In [25]:
import xlsxwriter
import csv
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import re

# path to input files

factoid_paths=["https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_1756er_Staatskalender_Meta_final_TEST-MB_FS0.xlsx",
               "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_1756er_Staatskalender_Meta_final_TEST-MB_FS1.xlsx",
               "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_1756er_Staatskalender_Meta_final_TEST-MB_FS2.xlsx",
               "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_1756er_Staatskalender_Meta_final_TEST-MB_FS3.xlsx",
               "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_1756er_Staatskalender_Meta_final_TEST-MB_FS4.xlsx",
               "https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/FactoidList_Erfassung_Erfurt_Master_2022-11-11_Kopie.xlsx"
               ]

# define dataframe for final output

f_to_add=[]

# structure of input files

# obligatory columns in valid factoid list

# read all data frames from path

frame_list=[]
for file in factoid_paths:
    df = pd.read_excel(file, index_col=None, dtype=str) # axis=1, sort=False sheet_name='FactoidList'
    df = df.fillna("n/a") # replace empty fields for string
    df_length=len(df)
    frame_list.append(df)

f = pd.concat(frame_list, axis=0, ignore_index=True, sort=False)

print("There are ", len(f), "items in your DataFrame!")

# delete all duplicate rows with exact matches

f_unique=f.drop_duplicates()
print("Your DataFrame has now ", len(f_unique), "items with at least one unique cell." )

# add columns missing according to factoid model

column_names = ["factoid_ID",
                "pers_ID",
                "pers_name",
                "alternative_names",
                "event_type",
                "event_after-date",
                "event_before-date",
                "event_start",
                "event_end",
                "event_date",
                "pers_title",
                "pers_function",
                "place_name",
                "inst_name",
                "rel_pers",
                "source_quotations",
                "additional_info",
                "comment",
                "info_dump",
                "source_combined",
                "event_value", # add more potential categorisations if needed
                "source",
                "source_site"]

df2 = f_unique.reindex(columns=column_names)

# populate some of the empty columns with data

df2.loc[:, "event_end"] = df2["event_start"]
#df2.loc[:, "event_type"] = ["Funktionsausübung"] * 37400 # add new column with standard event if column is completely absent!
df2["event_type"] = df2["event_type"].replace({'n/a':'Funktionsausübung'}) # add standard event in places where no other event is indicated!
df2.fillna('n/a', inplace=True) # fill remaining blanks with string to ensure that all cells can be processed in the same way!
df2['source_combined'] = df2['source'].astype(str) + ': ' + df2['source_site'].astype(str)

print("Done.")

# rename dataframe for next step

display(df2)

display(df2)


There are  40080 items in your DataFrame!
Your DataFrame has now  37400 items with at least one unique cell.
Done.


,factoid_ID,pers_ID,pers_name,alternative_names,event_type,event_after-date,event_before-date,event_start,event_end,event_date,...,inst_name,rel_pers,source_quotations,additional_info,comment,info_dump,source_combined,event_value,source,source_site
0,Stk_00010,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1739-00-00,1739-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1740B: 44,n/a,Stk_1740B,44
1,Stk_00052,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1740-00-00,1740-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1741: 45,n/a,Stk_1741,45
2,Stk_00087,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1741-00-00,1741-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1742: 38,n/a,Stk_1742,38
3,Stk_00125,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1742-00-00,1742-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1743: 37,n/a,Stk_1743,37
4,Stk_00170,9,Johann Werner von Vorster,n/a,Funktionsausübung,n/a,n/a,1744-00-00,1744-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat / S. 33,n/a,n/a,n/a,Stk_1745: 39,n/a,Stk_1745,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40075,Erffurt_006271,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Weihe,n/a,n/a,1788-02-03,1788-02-03,n/a,...,n/a,n/a,n/a,n/a,n/a,n/a,Gatz1983:Dalberg: 110,n/a,Gatz1983:Dalberg,110
40076,Erffurt_006272,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Weihe,n/a,n/a,1788-08-31,1788-08-31,n/a,...,n/a,n/a,Ernennung zum Titularerzbischof von Tarsus am ...,n/a,n/a,n/a,Gatz1983:Dalberg: 110,n/a,Gatz1983:Dalberg,110
40077,Erffurt_006273,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Funktionsausübung_Beginn,n/a,n/a,1810,1810,n/a,...,Großherzogtum Frankfurt,n/a,"""ohne Beziehung zu seiner geistlichen Würde""",n/a,n/a,n/a,Gatz1983:Dalberg: 112,n/a,Gatz1983:Dalberg,112
40078,Erffurt_006274,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Funktionsausübung_Ende,n/a,1814,n/a,n/a,n/a,...,Großherzogtum Frankfurt,n/a,n/a,n/a,n/a,n/a,Gatz1983:Dalberg: 112,n/a,Gatz1983:Dalberg,112


,factoid_ID,pers_ID,pers_name,alternative_names,event_type,event_after-date,event_before-date,event_start,event_end,event_date,...,inst_name,rel_pers,source_quotations,additional_info,comment,info_dump,source_combined,event_value,source,source_site
0,Stk_00010,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1739-00-00,1739-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1740B: 44,n/a,Stk_1740B,44
1,Stk_00052,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1740-00-00,1740-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1741: 45,n/a,Stk_1741,45
2,Stk_00087,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1741-00-00,1741-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1742: 38,n/a,Stk_1742,38
3,Stk_00125,9,Johann Werner von Vorstern,n/a,Funktionsausübung,n/a,n/a,1742-00-00,1742-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat,n/a,n/a,n/a,Stk_1743: 37,n/a,Stk_1743,37
4,Stk_00170,9,Johann Werner von Vorster,n/a,Funktionsausübung,n/a,n/a,1744-00-00,1744-00-00,n/a,...,"Hofrat / Regierung, adlige Bank",n/a,Hof- und Regierungsrat / S. 33,n/a,n/a,n/a,Stk_1745: 39,n/a,Stk_1745,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40075,Erffurt_006271,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Weihe,n/a,n/a,1788-02-03,1788-02-03,n/a,...,n/a,n/a,n/a,n/a,n/a,n/a,Gatz1983:Dalberg: 110,n/a,Gatz1983:Dalberg,110
40076,Erffurt_006272,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Weihe,n/a,n/a,1788-08-31,1788-08-31,n/a,...,n/a,n/a,Ernennung zum Titularerzbischof von Tarsus am ...,n/a,n/a,n/a,Gatz1983:Dalberg: 110,n/a,Gatz1983:Dalberg,110
40077,Erffurt_006273,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Funktionsausübung_Beginn,n/a,n/a,1810,1810,n/a,...,Großherzogtum Frankfurt,n/a,"""ohne Beziehung zu seiner geistlichen Würde""",n/a,n/a,n/a,Gatz1983:Dalberg: 112,n/a,Gatz1983:Dalberg,112
40078,Erffurt_006274,19,Karl Theodor Freiherr von Dalberg,Karl Theodor Anton Maria Reichsfreiherr von Da...,Funktionsausübung_Ende,n/a,1814,n/a,n/a,n/a,...,Großherzogtum Frankfurt,n/a,n/a,n/a,n/a,n/a,Gatz1983:Dalberg: 112,n/a,Gatz1983:Dalberg,112


In [26]:
# Merge input dataframe with dfs containing person IDs and geocoding

## Read person IDs from Github
## columns: pers_ID_MB, pers_name, alternative_names, Unnamed: 4, name_new_fs, pers_ID_FS,poi, pers_name_corr, freq
infile1="https://github.com/ieg-dhr/DigiKAR/raw/main/OntologyFiles/Factoid_PersonNames_merged.xlsx" # has to contain pers_name column!
person_df = pd.read_excel(infile1)

## Read geocoding from Github
infile2="https://github.com/ieg-dhr/DigiKAR/raw/main/OntologyFiles/Ortsontologie_Geocoded_gepr%C3%BCft.xlsx" # has to contain place_name column!
geo_df = pd.read_excel(infile2)

## Merge input dataframe horizontally

from functools import reduce

# define list of DataFrames
df_geocoded = [df2, geo_df] # dataframe list can be extended if necessary

# merge all DataFrames into one
merged_df1 = reduce(lambda  left,right: pd.merge(left,right,on=['place_name']),df_geocoded) # how='outer'

# define list of DataFrames
df_withIDs = [merged_df1, person_df] # dataframe list can be extended if necessary

# merge all DataFrames into one
merged_df2 = reduce(lambda  left,right: pd.merge(left,right,on=['pers_name', "alternative_names"]),df_withIDs) # how='outer'

display(merged_df2)

## Write new table to excel file (optional)

#outfile=directory+"AP3_final-df.xlsx"
#final_df.to_excel(outfile)

,factoid_ID,pers_ID,pers_name,alternative_names,event_type,event_after-date,event_before-date,event_start,event_end,event_date,...,Unnamed: 21,Unnamed: 22,NaN,pers_ID_MB,Unnamed: 4,name_new_fs,pers_ID_FS,poi,pers_name_corr,freq
0,Stk_00010,9,Johann Werner Freiherr von Vorster,Johann Werner von Vorster,n/a,n/a,n/a,1739-00-00,1739-00-00,n/a,...,NaN,NaN,1.734588e-04,NaN,NaN,NaN,NaN,NaN,Johann Werner Freiherr von Vorster,8
1,Stk_00052,9,Johann Werner Freiherr von Vorster,Johann Werner von Vorster,n/a,n/a,n/a,1740-00-00,1740-00-00,n/a,...,NaN,NaN,1.734588e-04,NaN,NaN,NaN,NaN,NaN,Johann Werner Freiherr von Vorster,8
2,Stk_00087,9,Johann Werner Freiherr von Vorster,Johann Werner von Vorster,n/a,n/a,n/a,1741-00-00,1741-00-00,n/a,...,NaN,NaN,1.734588e-04,NaN,NaN,NaN,NaN,NaN,Johann Werner Freiherr von Vorster,8
3,Stk_00125,9,Johann Werner Freiherr von Vorster,Johann Werner von Vorster,n/a,n/a,n/a,1742-00-00,1742-00-00,n/a,...,NaN,NaN,1.734588e-04,NaN,NaN,NaN,NaN,NaN,Johann Werner Freiherr von Vorster,8
4,Stk_00170,9,Johann Werner Freiherr von Vorster,Johann Werner von Vorster,n/a,n/a,n/a,1744-00-00,1744-00-00,n/a,...,NaN,NaN,1.734588e-04,NaN,NaN,NaN,NaN,NaN,Johann Werner Freiherr von Vorster,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,Erffurt_001193,255,Joachim Gerstenberger,Gerstenberg,Geburt,n/a,n/a,1593-04-14,1593-04-14,n/a,...,NaN,NaN,-1.363221e-05,255,NaN,Joachim Gerstenberger,255,poi-2,Joachim Gerstenberger,16
2016,Erffurt_002984,237,Johann Christian Freiherr von Boyneburg,Boineburg,Geburt,n/a,n/a,1622-04-12,1622-04-12,n/a,...,NaN,NaN,-2.213632e-05,237,NaN,Johann Christian von Boineburg,237,poi-2,Johann Christian Freiherr von Boyneburg,1
2017,Erffurt_003491,311,Georg Melchior von Ludolff,Leutholf,Funktionsausübung,n/a,n/a,1711,1711,n/a,...,NaN,NaN,4.820273e-05,311,NaN,Georg Melchior von Ludolph,311,poi-2,Georg Melchior von Ludolff,6
2018,Erffurt_002153,359,Conrad Theodoricus Brandt,Dietrich,Taufe,n/a,n/a,1616-10-01,1616-10-01,n/a,...,NaN,NaN,1.798165e-04,359,NaN,0,359,0,Conrad Theodoricus Brandt,2


In **step 2**, we reconstruct end dates for successive start dates. The data are automatically aggregated using Python's `groupby` function. If the results are too narrow or too broad, please change the aggregation rules below!


In [27]:
# Group the dataframe and aggregate the start and end dates
# code updated after problem with merged columns
# see discussion on Stackoverflow: https://stackoverflow.com/questions/76558443/column-remains-empty-when-using-map-with-dictionary-in-pandas-dataframe/76558586#76558586

grouped_df = merged_df2.groupby(['pers_ID_FS', 'event_type', "pers_function", "pers_title", "inst_name", "place_name"], as_index=False).agg(
                                                         {'event_start': 'min',
                                                          "event_after-date":'min',
                                                          "event_before-date":'max',
                                                          "event_end":'max',
                                                          "factoid_ID":list,
                                                          "pers_ID_MB":list,
                                                          "pers_name":list,
                                                          "alternative_names":list,
                                                          "Unnamed: 4":list,
                                                          "name_new_fs":list,
                                                          "pers_name_corr":list,
                                                          "rel_pers":list,
                                                          "source_quotations":list,
                                                          "additional_info":list,
                                                          "comment":list,
                                                          "info_dump":list,
                                                          "source_combined":list,
                                                          "event_value":list,
                                                          "place_name":list,
                                                          "address":list,
                                                          "addresses_full":list,
                                                          "ids":list,
                                                          "geonames address":list,
                                                          "latitudes":list,
                                                          "longitudes":list,
                                                          "lat":list,
                                                          "lng":list,
                                                          "Google address":list
                                                          })

display(grouped_df)

,pers_ID_FS,event_type,pers_function,pers_title,inst_name,event_start,event_after-date,event_before-date,event_end,factoid_ID,...,place_name,address,addresses_full,ids,geonames address,latitudes,longitudes,lat,lng,Google address
0,100,Funktionsausübung,Kammerdirektor],n/a,Regierung zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,[Erffurt_000358],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
1,100,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,[Erffurt_000340],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
2,100,Funktionsausübung,Kammerrat,n/a,[Kammer zu Erfurt],1692-00-00,n/a,n/a,1692-00-00,[Erffurt_000382],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
3,100,Funktionsausübung,Vizedomamtsrat [?],Kammerdirektor,Vizedomamt zu Erfurt,1679-00-00,n/a,n/a,1679-00-00,[Erffurt_000346],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
4,102,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1692-00-00,n/a,n/a,1699-00-00,"[Erffurt_000341, Erffurt_000383]",...,"[Erfurt, Erfurt]","[Erfurt, Erfurt]","[Erfurt, Europe, Erfurt, Europe]","[2929670, 2929670]","[Erfurt, Erfurt]","[50.9787, 50.9787]","[11.03283, 11.03283]","[50.98476789999999, 50.98476789999999]","[11.0298799, 11.0298799]","[Erfurt, Germany, Erfurt, Germany]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,96,Immatrikulation,n/a,n/a,Universität Erfurt,1737,n/a,n/a,1737,[Erffurt_004105],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
372,97,Aufnahme,Bürger,n/a,Bürgerschaft zu Erfurt,1752-10-10,n/a,n/a,1752-10-10,[Erffurt_000316],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
373,97,Herkunft,n/a,n/a,n/a,0000,n/a,n/a,0000,[Erffurt_004108],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"
374,97,Immatrikulation,n/a,n/a,Universität Erfurt,1728,n/a,n/a,1728,[Erffurt_004107],...,[Erfurt],[Erfurt],"[Erfurt, Europe]",[2929670],[Erfurt],[50.9787],[11.03283],[50.98476789999999],[11.0298799],"[Erfurt, Germany]"


In **step 3**, we can flatten the information and only preserve unique information per cell.

In [28]:
# flatten data in dataframe cells

def flatten_list(cell):
    if isinstance(cell, list):
        unique_values = set(cell)
        return ', '.join(str(value) for value in unique_values)
    else:
        return str(cell)

# flatten all cells containing lists
df3 = grouped_df.applymap(flatten_list)

# show the flattened DataFrame
display(df3)

,pers_ID_FS,event_type,pers_function,pers_title,inst_name,event_start,event_after-date,event_before-date,event_end,factoid_ID,...,place_name,address,addresses_full,ids,geonames address,latitudes,longitudes,lat,lng,Google address
0,100,Funktionsausübung,Kammerdirektor],n/a,Regierung zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,Erffurt_000358,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
1,100,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,Erffurt_000340,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
2,100,Funktionsausübung,Kammerrat,n/a,[Kammer zu Erfurt],1692-00-00,n/a,n/a,1692-00-00,Erffurt_000382,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
3,100,Funktionsausübung,Vizedomamtsrat [?],Kammerdirektor,Vizedomamt zu Erfurt,1679-00-00,n/a,n/a,1679-00-00,Erffurt_000346,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
4,102,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1692-00-00,n/a,n/a,1699-00-00,"Erffurt_000383, Erffurt_000341",...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,96,Immatrikulation,n/a,n/a,Universität Erfurt,1737,n/a,n/a,1737,Erffurt_004105,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
372,97,Aufnahme,Bürger,n/a,Bürgerschaft zu Erfurt,1752-10-10,n/a,n/a,1752-10-10,Erffurt_000316,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
373,97,Herkunft,n/a,n/a,n/a,0000,n/a,n/a,0000,Erffurt_004108,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
374,97,Immatrikulation,n/a,n/a,Universität Erfurt,1728,n/a,n/a,1728,Erffurt_004107,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"


In **step 4**, we enrich the data, e.g. by adding event values from an external Python dictionary stored in Github.

In [29]:
## load external dictionary with EVENT VALUES
# following method 2 on https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

# importing the module
import requests
import ast

master = "https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/Data%20Categorisation/Event_value_dict.txt" # add Sven's new mapping
req = requests.get(master)
req = req.text
print(req)

# reconstructing the data as a dictionary
event_value_dict = ast.literal_eval(req)
print(type(event_value_dict))

# add event values from dict to data frame

try:
    test = event_value_dict["Aufschwörung"] # random test if valid dict
    print("Value for chosen key: ", test)
except:
    print("Invalid dict structure!")

df3['event_value'] = df3['event_type'].map(event_value_dict) # optional: na_action='ignore'

display(df3)

{
    "Amtsantritt": "K",
    "Aufenthalt": "M",
    "Beisetzung": "Z",
    "Bewerbung": "I",
    "Eheschließung": "P",
    "Entlassung": "R",
    "Erfolglose Bewerbung": "I",
    "Flucht": "M",
    "Funktionsausübung": "Q",
    "Geburt": "A",
    "Graduation": "G",
    "Haft": "M",
    "Immatrikulation": "C",
    "Konflikt": "M",
    "Konversion": "M",
    "Mitgliedschaft": "L",
    "Nicht-Ausübung": "N",
    "Nobilitierung": "S",
    "Ordination": "J",
    "Primäre Bildungsstation": "B",
    "Promotion": "H",
    "Prüfung": "F",
    "Resignation": "T",
    "Rezeption": "K",
    "Sonstiges": "M",
    "Studium": "D",
    "Taufe": "A",
    "Tod": "Y",
    "Verleihung eines Ehrentitels": "S",
    "Wahl": "K",
    "Weihe": "J",
    "Wohnsitznahme": "O",
    "Zulassung": "E"
   }
   

<class 'dict'>
Invalid dict structure!


,pers_ID_FS,event_type,pers_function,pers_title,inst_name,event_start,event_after-date,event_before-date,event_end,factoid_ID,...,place_name,address,addresses_full,ids,geonames address,latitudes,longitudes,lat,lng,Google address
0,100,Funktionsausübung,Kammerdirektor],n/a,Regierung zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,Erffurt_000358,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
1,100,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1699-00-00,n/a,n/a,1699-00-00,Erffurt_000340,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
2,100,Funktionsausübung,Kammerrat,n/a,[Kammer zu Erfurt],1692-00-00,n/a,n/a,1692-00-00,Erffurt_000382,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
3,100,Funktionsausübung,Vizedomamtsrat [?],Kammerdirektor,Vizedomamt zu Erfurt,1679-00-00,n/a,n/a,1679-00-00,Erffurt_000346,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
4,102,Funktionsausübung,Kammerrat,n/a,Kammer zu Erfurt,1692-00-00,n/a,n/a,1699-00-00,"Erffurt_000383, Erffurt_000341",...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,96,Immatrikulation,n/a,n/a,Universität Erfurt,1737,n/a,n/a,1737,Erffurt_004105,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
372,97,Aufnahme,Bürger,n/a,Bürgerschaft zu Erfurt,1752-10-10,n/a,n/a,1752-10-10,Erffurt_000316,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
373,97,Herkunft,n/a,n/a,n/a,0000,n/a,n/a,0000,Erffurt_004108,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"
374,97,Immatrikulation,n/a,n/a,Universität Erfurt,1728,n/a,n/a,1728,Erffurt_004107,...,Erfurt,Erfurt,"Erfurt, Europe",2929670,Erfurt,50.9787,11.03283,50.98476789999999,11.0298799,"Erfurt, Germany"


In [18]:
## load external dictionary with EVENT CATEGORIES (e.g. I: agent-oriented)
# following method 2 on https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

# importing the module
import requests
import ast

master = "https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/Data%20Categorisation/####.txt" # add file name
req = requests.get(master)
req = req.text
print(req)

# reconstructing the data as a dictionary
event_category_dict = ast.literal_eval(req)
print(type(event_category_dict))

# add event values from dict to data frame

try:
    test = event_category_dict["Geburt"] # random test if valid dict
    print("Value for chosen key: ", test)
except:
    print("Invalid dict structure!")

df3['event_category'] = df3['event_type'].map(event_category_dict) # optional: na_action='ignore'

display(df3)

400: Invalid request


SyntaxError: ignored

In [ ]:
## load external dictionary with FUNCTION CATEGORIES (e.g. teaching versus administration)
# following method 2 on https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

# importing the module
import requests
import ast

master = "https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/Data%20Categorisation/####.txt" # add file name
req = requests.get(master)
req = req.text
print(req)

# reconstructing the data as a dictionary
function_category_dict = ast.literal_eval(req)
print(type(function_category_dict))

# add event values from dict to data frame

try:
    test = function_category_dict["Professor"] # random test if valid dict
    print("Value for chosen key: ", test)
except:
    print("Invalid dict structure!")

df3['function_category'] = df3['pers_function'].map(function_category_dict) # optional: na_action='ignore'

display(df3)

In [30]:
# save enriched df to DRIVE

workbook=directory+'FACTOIDS_consolidated/Factoid_Staatskalender-Erfurt_ALL_consolidation_coordinates_event-values_person-IDs.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
df3.to_excel(writer, sheet_name='FactCons1') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")

/content/drive/My Drive/Colab_DigiKAR/FACTOIDS_consolidated/Factoid_Staatskalender-Erfurt_ALL_consolidation_coordinates_event-values_person-IDs.xlsx
Done.


<ipython-input-30-c7b491c7c644>:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save() # Close the Pandas Excel writer and output the Excel file.


Check the output files and repeat process if necessary.

Script by Monika Barget, Maastricht/Mainz

June 2023
